In [1]:
import os, sys
import vcgencmd as vc
import RPi.GPIO as gpio
from bottle   import Bottle, route, run, template
from datetime import datetime

In [2]:
# see https://pypi.org/project/vcgencmd/ for usage

In [3]:
dirname = os.path.dirname(sys.argv[0])
dirname

'/usr/lib/python3/dist-packages'

In [4]:
sys.argv

['/usr/lib/python3/dist-packages/ipykernel_launcher.py',
 '-f',
 '/home/pi/.local/share/jupyter/runtime/kernel-4b7301ce-54f6-45c3-960e-bcac493c3571.json']

In [5]:
dd = os.popen('pwd')
#dirname = os.path.dirname(dd.read())
pwd = dd.read()[:-1]
pwd

'/home/pi/Devel/raspberry_pi/rpi-jupyter/jupyter'

In [6]:
vccmd = vc.Vcgencmd()

In [7]:
vccmd.version()

'Dec  1 2021 15:02:46 \nCopyright (c) 2012 Broadcom\nversion 71bd3109023a0c8575585ba87cbb374d2eeb038f (clean) (release) (start_x)\n'

In [54]:
def get_temp():
    temp_C = vccmd.measure_temp()
    temp_F = temp_C * 9./5. +32.0
    return "CPU TEMP: {:0.2f} deg C, {:0.2f} deg F".\
            format(temp_C, temp_F)

def get_time():
    return "TIME: {:%Y-%m-%d %H:%M:%S}".format(datetime.now())

def get_load_average():
    one, five, fifteen = os.getloadavg()
    return "PROCESS AVERAGES: {} (1m), {} (5m), {} (15m)".\
            format(one, five, fifteen)

def get_uptime():
    dd = os.popen('/usr/bin/uptime -p')
    return "UPTIME: {}".format(dd.read()[3:-1])

def get_freq(obj='arm'):
    # obj = arm, core
    return "FREQ of '{}': {:0.3f} GHz".\
        format(obj, vccmd.measure_clock(obj)/1.e9)
'''
def get_processes(num=5): # this version returns a text block
    # see unix.stackexchange.com #13968 : sorting on cpu%
    # top result is the header, so get n + 1
    cmd = '/bin/ps aux --sort=-pcpu | head -n {}'.format(num + 1)
    dd  = os.popen(cmd)
    txt = dd.read()
    out = [k.split() for k in txt.split('\n')]
    fields = [0, 1, 2, 3, 8, 9, 10] # limit the output fields
    txt = ''
    for k in fields:
        txt += out[0][k] + '\t'
    for j in range(1, len(out) - 1):
        txt += '\n'
        for k in fields: # split affects only the last field
            txt += out[j][k].split('/')[-1] + '\t'
    return txt
'''
def get_processes(num=5): # this version returns a list of lists
    # see unix.stackexchange.com #13968 : sorting on cpu%
    # top result is the header, so get n + 1
    cmd = '/bin/ps aux --sort=-pcpu | head -n {}'.format(num + 1)
    dd  = os.popen(cmd)
    txt = dd.read()
    # [:-1] gets rid of empty list after last \n
    out = [k.split() for k in txt.split('\n')][:-1]
    fields = [0, 1, 2, 3, 8, 9, 10] # limit the output fields
    short = []
    for j in range(len(out)): # split affects only the last field
        short.append([out[j][k].split('/')[-1] for k in fields])
    return short

In [55]:
fns = [get_temp, get_time, get_load_average, get_uptime, get_freq]
dd1 = [k() for k in fns]
dd1

['CPU TEMP: 42.80 deg C, 109.04 deg F',
 'TIME: 2021-12-24 18:13:23',
 'PROCESS AVERAGES: 0.73 (1m), 0.58 (5m), 0.61 (15m)',
 'UPTIME: 3 days, 1 hour, 16 minutes',
 "FREQ of 'arm': 1.500 GHz"]

In [57]:
dd2 = get_processes(4)
dd2

[['USER', 'PID', '%CPU', '%MEM', 'START', 'TIME', 'COMMAND'],
 ['pi', '1274', '4.4', '2.5', 'Dec22', '126:47', 'chromium-browser-v7'],
 ['pi', '16687', '2.8', '0.2', 'Dec22', '90:08', 'lxtask'],
 ['root', '681', '2.2', '1.2', 'Dec21', '100:27', 'Xorg'],
 ['pi', '27952', '1.5', '1.8', 'Dec22', '49:15', 'chromium-browser-v7']]

In [ ]:
vccmd.get_sources?

In [ ]:
#get_sources("mem") will return ["arm", "gpu"] which are the sources/arguments for 
#get_mem(source) that returns the memory of the source passed as argument to it.
# the below dict from /usr/local/lib/python3.7/dist-packages/vcgencmd/vcgencmd.py
sources = {
  "clock": ["arm", "core","isp", "v3d", "uart", "pwm", "emmc", "pixel", 
            "vec", "hdmi", "dpi"], # deleted H264: gives error
  "volts": ["core", "sdram_c", "sdram_i", "sdram_p"],
  "mem":   ["arm", "gpu"],
  "codec": ["agif", "flac", "h263", "h264", "mjpa", "mjpb", "mjpg", "mpg2", 
            "mpg4", "mvc0", "pcm", "thra", "vorb", "vp6", "vp8", "wmv9", "wvc1"],
  "display_id": [0, 1, 2, 3, 7]
  }

In [ ]:
dd = vccmd.get_sources('mem')
for k in dd:
    print(k, vccmd.get_mem(k))

In [ ]:
dd = vccmd.get_sources('clock')
for k in dd:
    # H264 produces an error
    if (k != 'H264'): print(k, vccmd.measure_clock(k))

In [ ]:
dd = vccmd.get_sources('volts')
for k in dd:
    print(k, vccmd.measure_volts(k))

In [ ]:
dd = vccmd.get_sources('codec')
for k in dd:
    print(k, vccmd.codec_enabled(k))

In [ ]:
dd = vccmd.get_sources('display_id')
for k in dd:
    print(k, vccmd.display_power_state(k))

In [ ]:
vccmd.get_lcd_info()

In [ ]:
vccmd.dispmanx_list()

In [ ]:
#vccmd.otp_dump()

In [ ]:
#dev = os.popen('bin/ps -a', 'r', 1)
dev = os.popen('/opt/vc/bin/vcgencmd measure_temp')  # this works
dev.read()
#dev.close()

In [ ]:
os.getlogin()

In [ ]:
os.listdir()

In [ ]:
#os.sysconf_names # prints out a lot

# now play with bottle

In [ ]:
def index(name='RPi Status'):
    dt = get_time()
    tc = get_temp()
    info = {'name': name, 'dt': dt, 'tc': tc}
    return template('test.tpl', info)

In [ ]:
tt = index()
tt

In [ ]:
import sys
sys.path

In [ ]:
app = Bottle()